In [2]:
import pandas as pd
import numpy as np
import vectorbt as vbt 

In [5]:
df = pd.read_csv('BTCUSDT_2020-2025_mean_reversion/BTCUSDT_2020-2025_mean_reversion_combined_oos.csv', index_col='Timestamp', parse_dates=True)

In [6]:
df

,Unnamed: 0,Open,High,Low,Close,Volume,Regime,signal,exit,pair,window
Timestamp,,,,,,,,,,,
2020-12-17 23:00:00,1970-01-01 00:00:00.000000023,22974.98,23016.52,22592.43,22816.99,18662.046,Trend,0,0,BTCUSDT_2020-2025_mean_reversion,1
2020-12-18 00:00:00,1970-01-01 00:00:00.000000024,22816.99,22860.00,22475.47,22784.39,15747.879,Trend,0,1,BTCUSDT_2020-2025_mean_reversion,1
2020-12-18 01:00:00,1970-01-01 00:00:00.000000025,22784.39,23178.00,22655.52,23015.00,21091.456,Trend,0,1,BTCUSDT_2020-2025_mean_reversion,1
2020-12-18 02:00:00,1970-01-01 00:00:00.000000026,23015.00,23277.99,22960.32,23033.26,12104.044,Trend,0,1,BTCUSDT_2020-2025_mean_reversion,1
2020-12-18 03:00:00,1970-01-01 00:00:00.000000027,23036.10,23072.89,22781.77,22833.17,11712.415,Trend,0,-1,BTCUSDT_2020-2025_mean_reversion,1
...,...,...,...,...,...,...,...,...,...,...,...
2025-01-29 07:00:00,1970-01-01 00:00:00.000003606,102763.20,102768.20,102417.40,102723.20,3872.373,Trend,0,-1,BTCUSDT_2020-2025_mean_reversion,10
2025-01-29 08:00:00,1970-01-01 00:00:00.000003607,102723.20,103012.60,102644.10,102705.00,4329.903,Trend,0,-1,BTCUSDT_2020-2025_mean_reversion,10
2025-01-29 09:00:00,1970-01-01 00:00:00.000003608,102705.00,102744.80,102258.40,102300.00,4454.092,Trend,0,-1,BTCUSDT_2020-2025_mean_reversion,10


In [ ]:
# Shifting all signals and exits by 1 to avoid lookahead bias
long_entries = (df['signal'] == 1).shift(1).astype(bool) 
short_entries = (df['signal'] == -1).shift(1).astype(bool) 

exit_long_signal = (df['exit'] == 1).shift(1).astype(bool) 
exit_short_signal = (df['exit'] == -1).shift(1).astype(bool) 

pf = vbt.Portfolio.from_signals(
            df['Open'], # Open price used for entry
            entries=long_entries,
            exits=exit_long_signal,
            short_entries=short_entries,
            short_exits=exit_short_signal,
            freq='1h'
            fees=0.0005,
            init_cash=10000,
            accumulate=False,
            #upon_opposite_entry='ignore',
        )